## package install

```
$ pip install oandapyV20
$ pip install backtesting==0.1.7
$ pip install TA-Lib
$ brew install ta-lib
$ conda install pandas
$ conda install configparser
$ conda install requests
```

In [1]:
import configparser
import datetime
import pandas as pd
from oanda import Client
config = configparser.ConfigParser()
config.read("../oanda_config.txt")
api_key = config["Practice"]["api_key"]
oanda = Client(api_key, "", "practice")

In [3]:
UTC = datetime.timezone.utc
date_from = datetime.datetime(2020, 11, 3, 0, 0, tzinfo=UTC)
date_to = datetime.datetime(2020, 11, 9, 23, 55, tzinfo=UTC)
candles = oanda.get_candles("USD_JPY", date_from, date_to, "M5")
print(candles)

V20Error: {"errorMessage":"Maximum value for 'count' exceeded"}

In [ ]:
from backtesting import Strategy
from backtesting.lib import crossover
import talib

def EMA_Backtesting(values, n):
    """
    Return exponential moving average of `values`, at
    each step taking into account `n` previous values.
    """
    close = pd.Series(values)
    return talib.EMA(close, timeperiod=n)

class EmaCrossStrategy(Strategy):

    # Define the two EMA lags as *class variables*
    # for later optimization
    n1 = 5
    n2 = 10

    def init(self):
        # Precompute two moving averages
        self.ema1 = self.I(EMA_Backtesting, self.data.Close, self.n1)
        self.ema2 = self.I(EMA_Backtesting, self.data.Close, self.n2)

    def next(self):
        # If ema1 crosses above ema2, buy the asset
        if crossover(self.ema1, self.ema2):
            self.position.close()
            self.buy()

        # Else, if ema1 crosses below ema2, sell it
        elif crossover(self.ema2, self.ema1):
            self.position.close()
            self.sell()

In [ ]:
from backtesting import Backtest

candles_ = candles.copy()
bt_ema = Backtest(candles_, EmaCrossStrategy, cash=1, commission=0, trade_on_close=True)
bt_ema.run()
bt_ema.plot()